In [ ]:
import cv2
import numpy as np
import glob
import pickle

#read here https://opencv24-python-tutorials.readthedocs.io/en/stable/py_tutorials/py_calib3d/py_calibration/py_calibration.html

# Define the dimensions of checkerboard
CHECKERBOARD = (6, 9)

# Define side length
checkerDimension = 0.01882 #meter

# stop the iteration when specified
# accuracy, epsilon, is reached or
# specified number of iterations are completed.
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# Vector for 3D points
object_points = []
# Vector for 2D points
image_points = []

#  3D points real world coordinates
objectp3d = np.zeros((CHECKERBOARD[0] * CHECKERBOARD[1],3), np.float32)
objectp3d[:,:2] = np.mgrid[0:CHECKERBOARD[0],
                  0:CHECKERBOARD[1]].T.reshape(-1, 2)

# Extracting path of individual image stored
# in a given directory. Since no path is
# specified, it will take current directory
# jpg files alone
images = glob.glob('calibrationImages/iPhone8/*.JPG')
for filename in images:
    image = cv2.imread(filename)
    grayColor = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Find the chess board corners
    # If desired number of corners are
    # found in the image then ret = true
    ret, corners = cv2.findChessboardCorners(
        grayColor,
        CHECKERBOARD,
        cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_NORMALIZE_IMAGE)

    # If desired number of corners can be detected then,
    # refine the pixel coordinates
    if ret == True:
        object_points.append(objectp3d)
        # Refining pixel coordinates
        # for given 2d points.
        print(filename)
        corners2 = cv2.cornerSubPix(grayColor, corners, (11, 11), (-1, -1), criteria)

        image_points.append(corners2)

    # Draw and display the corners leave commented if not wanted
    # image = cv2.drawChessboardCorners(image,CHECKERBOARD,corners2, ret)
    #
    # imS = cv2.resize(image, (1008, 756))
    # cv2.imshow('img', imS)
    # cv2.waitKey(0)

#cv2.destroyAllWindows()

#normalize the object points to the real edge length of a checker and convert to np.array
object_points = np.array(object_points)*checkerDimension
image_points = np.array(image_points)

# Perform camera calibration by
# passing the value of above found out 3D points (object_points)
# and its corresponding pixel coordinates of the
# detected corners (image_points)
ret, k_matrix, distortion, r_vecs, t_vecs = cv2.calibrateCamera(object_points, image_points, grayColor.shape[::-1], None, None)

#safe the determined calibration parameter in a file, to use them later
np.save('cam_parameters_iPhone8.npy', (object_points, image_points, ret, k_matrix, distortion, np.array(r_vecs), np.array(t_vecs)))